In [4]:
import mvpa2
import pandas as pd
import numpy as np
import sklearn
#to manipulate data
from mvpa2.datasets import *
import scipy.io

In [5]:
#makes a dict
matFile = scipy.io.loadmat('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/mem/MSC02_parcel_corrmat.mat')
#convert to numpy array
task_FC=np.array(matFile['parcel_corrmat'])

In [40]:
#Index of upper triangle of matrix 
nrois=333
nsess=10
mask=np.triu_indices(nrois, 1)
corrlin=np.empty((nsess, int(nrois*(nrois-1)/2)))
count=0
#loop through all 10 days to reshape correlations into linear form
for sess in range(nsess):
    tmp=task_FC[:,:,sess]
    corrlin[count]=tmp[mask]
    count=count+1

In [42]:
corrlin.shape

(10, 55278)

In [43]:
ds=Dataset(corrlin)

In [44]:
ds.shape

(10, 55278)

In [45]:
ds.nfeatures

55278

In [47]:
ds.nsamples

10

In [52]:
ds.sa['Task']=

ValueError: ArrayCollectable only takes sequences as value.